## Desafío - Clasificación desde la econometría


<br>
<br>

Nombre: **Nicolás Gómez**

Profesor: Gabriel Tamayo

Generación: **G2**


* Para poder realizar esta actividad debes haber revisado la lectura y videos correspondiente a la unidad.
* Crea una carpeta de trabajo y guarda todos los archivos correspondientes (notebook y csv).
* Una vez terminado el desafío, comprime la carpeta y sube el .zip

### Descripción


En esta sesión trabajaremos con:
* sbp : Presión Sanguínea Sistólica.
* tobacco : Promedio tabaco consumido por día.
* ldl : Lipoproteína de baja densidad.
* adiposity : Adiposidad.
* famhist : Antecedentes familiares de enfermedades cardiácas. (Binaria)
* types : Personalidad tipo A
* obesity : Obesidad.
* alcohol : Consumo actual de alcohol.
* age : edad.
* chd : Enfermedad coronaria. (dummy)


### Desafío 1: Preparar el ambiente de trabajo

* Importe las librerías básicas para el análisis de datos.
* Incluya los import base de statsmodels .
* Importe la base de datos southafricanheart.csv  y elimine la columna 'Unnamed: 0'

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns

dataframe = pd.read_csv('southafricanheart.csv')
dataframe = dataframe.drop('Unnamed: 0', axis = 1)
dataframe.head(5)

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,160,12.00,5.73,23.11,Present,49,25.30,97.20,52,1
1,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
2,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
3,170,7.50,6.41,38.03,Present,51,31.99,24.26,58,1
4,134,13.60,3.50,27.78,Present,60,25.99,57.34,49,1


### Desafío 2
A continuación se presenta el siguiente modelo a estimar:

$$\log\left(\frac{p({ chd=1})}{1 - p({ chd=1})}\right)={{\beta }_{0} + {\beta }_{1} \cdot {famhist}}$$


Para ello ejecute los siguientes pasos:

1. Recodifique famhist a dummy, asignando 1 a la categoría minoritaria.
2. Utilice smf.logit para estimar el modelo.
3. Implemente una función inverse_logit que realize el mapeo de log-odds a probabilidad.
4. Con el modelo estimado, responda lo siguiente:
    * ¿Cuál es la probabilidad de un individuo con antecedentes familiares de tener una enfermedad coronaria?
    * ¿Cuál es la probabilidad de un individuo sin antecedentes familiares de tener una enfermedad coronaria?
    * ¿Cuál es la diferencia en la probabilidad entre un individuo con antecedentes y otro sin antecedentes?

Replique el modelo con smf.ols y comente las similitudes entre los coeficientes estimados.

grafique ambas curvas con lmplot



In [2]:
dataframe['famhist'].value_counts()

Absent     270
Present    192
Name: famhist, dtype: int64

In [3]:
# El minoritario es "Present"
df = pd.get_dummies(dataframe).drop('famhist_Present', axis = 1)
df.head(5)

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,chd,famhist_Absent
0,160,12.00,5.73,23.11,49,25.30,97.20,52,1,0
1,144,0.01,4.41,28.61,55,28.87,2.06,63,1,1
2,118,0.08,3.48,32.28,52,29.14,3.81,46,0,0
3,170,7.50,6.41,38.03,51,31.99,24.26,58,1,0
4,134,13.60,3.50,27.78,60,25.99,57.34,49,1,0


In [4]:
model_logit = smf.logit('chd ~ famhist_Absent', df).fit()

Optimization terminated successfully.
         Current function value: 0.608111
         Iterations 5


In [5]:
def invlogit(x):
    return 1 / (1 + np.exp(-x))

In [6]:
model_logit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      460
Method:                           MLE   Df Model:                            1
Date:                Mon, 08 Jul 2019   Pseudo R-squ.:                 0.05740
Time:                        20:55:11   Log-Likelihood:                -280.95
converged:                       True   LL-Null:                       -298.05
                                        LLR p-value:                 4.937e-09
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept      -3.586e-16      0.144  -2.48e-15      1.000      -0.283       0.283
famhist_Absent    -1.1690      0.203     -5.751      0.000      -1.567      -0.771
==================================================================================
"""

In [21]:
print("¿Cuál es la probabilidad de un individuo con antecedentes familiares de tener una enfermedad coronaria?")
p_with_famhist = invlogit(model_logit.params['Intercept'] + model_logit.params['famhist_Absent'] * 1)
round(p_with_famhist, 2)

¿Cuál es la probabilidad de un individuo con antecedentes familiares de tener una enfermedad coronaria?


0.24

In [22]:
print("¿Cuál es la probabilidad de un individuo sin antecedentes familiares de tener una enfermedad coronaria?")
p_without_famhist = invlogit(model_logit.params['Intercept'] + model_logit.params['famhist_Absent'] * 0)
round(p_without_famhist, 2)

¿Cuál es la probabilidad de un individuo sin antecedentes familiares de tener una enfermedad coronaria?


0.5

In [23]:
print("¿Cuál es la diferencia en la probabilidad entre un individuo con antecedentes y otro sin antecedentes?")
round(abs(p_with_famhist - p_without_famhist), 2)

¿Cuál es la diferencia en la probabilidad entre un individuo con antecedentes y otro sin antecedentes?


0.26

### Desafío 3

Implemente un modelo con la siguiente forma

$$\log\left(\frac{p({ chd=1})}{1 - p({ chd=1})}\right)={{\beta }_{0} + \sum_{j=0}^N {\beta }_{j} \cdot {X}}$$



1. Depure el modelo manteniendo las variables con significancia estadística al 95%.
2. Reporte de forma sucinta el efecto de las variables en el log-odds de tener una enfermedad coronaria.


In [10]:
def generate_model(y, attributes, data):
    return smf.logit('{} ~ {}'.format(y, ' + '.join(attributes)), df).fit()

columns = filter(lambda col: col != 'chd', [x for x in df.columns])
model = generate_model('chd', columns, df)

Optimization terminated successfully.
         Current function value: 0.510974
         Iterations 6


In [11]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      452
Method:                           MLE   Df Model:                            9
Date:                Mon, 08 Jul 2019   Pseudo R-squ.:                  0.2080
Time:                        20:55:11   Log-Likelihood:                -236.07
converged:                       True   LL-Null:                       -298.05
                                        LLR p-value:                 2.055e-22
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -5.2254      1.315     -3.973      0.000      -7.803      -2.647
sbp                0.0065      0.006      1.135      0.256      -0.005       0.018
tobacco            0.0794      0.027      2.984      0.003       0.027       0.132
ldl                0.1739      0.060      2.915      0.004       0.057       0.291
adiposity          0.0186      0.029      0.635      0.526      -0.039       0.076
typea              0.0396      0.012      3.214      0.001       0.015       0.064
obesity           -0.0629      0.044     -1.422      0.155      -0.150       0.024
alcohol            0.0001      0.004      0.027      0.978      -0.009       0.009
age                0.0452      0.012      3.728      0.000       0.021       0.069
famhist_Absent    -0.9254      0.228     -4.061      0.000      -1.372      -0.479
==================================================================================
"""

In [12]:
# ¿Cuáles no cumplen al 95%?
model.pvalues > 0.025

Intercept         False
sbp                True
tobacco           False
ldl               False
adiposity          True
typea             False
obesity            True
alcohol            True
age               False
famhist_Absent    False
dtype: bool

En base a la tabla, tenemos que descartar aquellos donde el p value es > 0.025, es decir, descartamos: sbp, adiposity, obesity, alcohol

In [13]:
model = generate_model('chd', ['tobacco', 'ldl', 'typea', 'age', 'famhist_Absent'], df)

Optimization terminated successfully.
         Current function value: 0.514811
         Iterations 6


In [14]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      456
Method:                           MLE   Df Model:                            5
Date:                Mon, 08 Jul 2019   Pseudo R-squ.:                  0.2020
Time:                        20:55:12   Log-Likelihood:                -237.84
converged:                       True   LL-Null:                       -298.05
                                        LLR p-value:                 2.554e-24
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -5.5383      0.928     -5.969      0.000      -7.357      -3.720
tobacco            0.0804      0.026      3.106      0.002       0.030       0.131
ldl                0.1620      0.055      2.947      0.003       0.054       0.270
typea              0.0371      0.012      3.051      0.002       0.013       0.061
age                0.0505      0.010      4.944      0.000       0.030       0.070
famhist_Absent    -0.9082      0.226     -4.023      0.000      -1.351      -0.466
==================================================================================
"""

In [15]:
def calc_prob(model_params, values):
    value = model_params['Intercept']
    for attr in values:
        value += model_params[attr] * values[attr]
    return invlogit(value)

means = df.mean()

p1 = calc_prob(model.params, {
    'tobacco': means['tobacco'],
    'ldl': means['ldl'],
    'typea': means['typea'],
    'age': means['age'],
    'famhist_Absent': 1,
})

p2 = calc_prob(model.params, {
    'tobacco': means['tobacco'],
    'ldl': means['ldl'],
    'typea': means['typea'],
    'age': means['age'],
    'famhist_Absent': 0,
})

In [24]:
round(p1, 2)

0.22

In [25]:
round(p2, 2)

0.41

Podemos ver que el promedio esperado para cuando no hay antecedentes familiares es de ~22%, mientras que el promedio esperado cuando hay antecedentes familiares es de 41%, casi el doble.

### Desafío 4: Estimación de perfiles
A partir del modelo depurado, genere las estimaciones en log-odds y posteriormente transfórmelas
a probabilidades con inverse_logit . Los perfiles a estimar son los siguientes:
* La probabilidad de tener una enfermedad coronaria para un individuo con características similares a la muestra.
* La probabilidad de tener una enfemerdad coronaria para un individuo con altos niveles de lipoproteína de baja densidad, manteniendo todas las demás características constantes.
* La probabilidad de tener una enfemerdad coronaria para un individuo con bajos niveles de lipoproteína de baja densidad, manteniendo todas las demás características constantes.

In [28]:
print('La probabilidad de tener una enfermedad coronaria para un individuo con características similares a la muestra')
# Esto ya se calculó en la parte 3, es igual a p1 (promedios con Absent = true)
round(p1, 2)

La probabilidad de tener una enfermedad coronaria para un individuo con características similares a la muestra


0.22

In [29]:
print('La probabilidad de tener una enfemerdad coronaria para un individuo con altos niveles de lipoproteína de baja densidad, manteniendo todas las demás características constantes.')

round(calc_prob(model.params, {
    'tobacco': means['tobacco'],
    'ldl': df['ldl'].max(),
    'typea': means['typea'],
    'age': means['age'],
    'famhist_Absent': 1,
}), 2)

La probabilidad de tener una enfemerdad coronaria para un individuo con altos niveles de lipoproteína de baja densidad, manteniendo todas las demás características constantes.


0.61

In [30]:
print('La probabilidad de tener una enfemerdad coronaria para un individuo con bajos niveles de lipoproteína de baja densidad, manteniendo todas las demás características constantes.')

round(calc_prob(model.params, {
    'tobacco': means['tobacco'],
    'ldl': df['ldl'].min(),
    'typea': means['typea'],
    'age': means['age'],
    'famhist_Absent': 1,
}), 2)

La probabilidad de tener una enfemerdad coronaria para un individuo con bajos niveles de lipoproteína de baja densidad, manteniendo todas las demás características constantes.


0.13

En base a esto, el sujeto medio tendrá una probabilidad del 22% de tener una enfermedad coronaria.

Una persona con niveles bajos de _lipoproteína de baja densidad_ (`ldl`) tendrá solo un 13% de probabilidad de tener una enfermedad coronaria (menor que el sujeto promedio), mientras que la probabilidad de alguien con una alta `ldl` es del 61% (4 veces más!)

¿Qué ocurre con otras variables?

In [33]:
for var in ['tobacco', 'typea', 'age']:
    print('--', var, '--')
    
    values = {
        'tobacco': means['tobacco'],
        'ldl': means['ldl'],
        'typea': means['typea'],
        'age': means['age'],
        'famhist_Absent': 1,
    }

    values[var] = df[var].max()
    print('Max:\t', round(calc_prob(model.params, values), 2))
    
    values[var] = df[var].min()
    print('Min:\t', round(calc_prob(model.params, values), 2))

    print('\n')


-- tobacco --
Max:	 0.72
Min:	 0.18


-- typea --
Max:	 0.42
Min:	 0.06


-- age --
Max:	 0.45
Min:	 0.07




Podemos ver que el Tabaco consumido por día influye fuertemente en el promedio (manteniendo las demás variables como si fuera un sujeto promedio).

La probabilidad de la persona que menos fuma parte en 18% (menor sujeto promedio), mientras que la probabilidad para la persona que fuma más es de 72% (3.2 veces el promedio) 🙃